In [61]:
import re
import numpy as np

target_function = "1*x1 +2*x2"
constraint_functions = "-3*x1 +2*x2 <= 2\n-1*x1 +2*x2 <= 4\n1*x1 +1*x2<= 5"

constraint_simple = "x1 >= 0\nx2 >= 0"

maximisation = True

In [33]:
def target_split(target_funct):
    coeff_regex = r'([+-]?\d+)\s*\*?\s*([a-zA-Z0-9]+)\s*'
    signe_regex = r'([<>]=?|==)\s*([+-]?\d+)'
    target_catcher = re.findall(coeff_regex, target_funct)
    if maximisation:
        target_coeff = [-int(match[0]) for match in target_catcher]
    else:
        target_coeff = [int(match[0]) for match in target_catcher]
        
    return target_coeff

def variables(target_funct):
    var_regex = r'([+-]?\d+)\s*\*?\s*([a-zA-Z0-9]+)\s*'
    variable_catcher = re.findall(var_regex, target_funct)
    target_var = [match[1] for match in variable_catcher]
    return target_var
    

In [34]:
def constraint_split(constraints):
    constraint_one_by_one = constraints.split('\n')
    coeff_regex = r'([+-]?\d+)\s*\*?\s*([a-zA-Z0-9]+)\s*'
    signe_regex = r'([<>]=?|==)\s*([+-]?\d+)'
    coeff_tab = []
    variables_tab = []
    signe_tab = []
    const_tab = []
    
    for constraint in constraint_one_by_one:
        coeff_catcher = re.findall(coeff_regex, constraint)
        signe_catcher = re.findall(signe_regex, constraint)
        coefficients = [int(match[0]) for match in coeff_catcher]
        variables = [match[1] for match in coeff_catcher]
        coeff_tab.append(coefficients)
        variables_tab.append(variables)
        signe = [match[0] for match in signe_catcher]
        #signe = signe_catcher[0]

        constant = [int(match[1]) for match in signe_catcher]
        signe_tab.append(signe)
        const_tab.append(constant)
        
    output = {
        "coefficients": coeff_tab,
        "variables": variables_tab,
        "signes": signe_tab,
        "constantes": const_tab
    }
    return output
    

In [35]:
#print(signe_tab[2][0])
#A = np.array(coeff_tab)
#print("A = \n",A)

def count_sign(tab):
    cpt=0
    for sign in tab:
        if sign[0] == '<=' or sign[0] == '>=' or sign[0] == '>' or sign[0] == '<':
            cpt = cpt + 1
    return cpt

def sign_number(constraints):
    constraint = constraint_split(constraints)
    nbr_sign = count_sign(constraint["signes"])
    
    return nbr_sign

def add_zero(M, nbr_col):
    number_rows = M.shape[0]
    zero_array = np.zeros((number_rows, nbr_col))
    new_array = np.hstack((M, zero_array))
    return new_array

def constraint_standard_form(constraints):
    constraint = constraint_split(constraints)
    A = np.array(constraint["coefficients"])
    constantes = constraint["constantes"]
    variable = np.unique(constraint["variables"])
    nbr_sign = count_sign(constraint["signes"])
    
    new_arr = add_zero(A, nbr_sign)
    
    for a in enumerate(constraint["signes"]):
        print(a[1][0])
        #i = signe_tab.index(a)
        i = a[0]
        print("i = ",i)
        if a[1][0] == '<' or a[1][0] == '<=':
            new_arr[i][-nbr_sign] = 1
            nbr_sign -= 1
        elif a[1][0] == '>' or a[1][0] == '>=':
            new_arr[i][-nbr_sign] = -1
            nbr_sign -= 1
        print(new_arr[i])

    resp = {
        "matrix": new_arr,
        "constantes":constantes
    }
    
    return resp
        

In [36]:
def target_standard_form(target_funct, constraint):
    nbr = sign_number(constraint)
    target = target_split(target_funct)
    target += [0] * nbr
    
    return target

In [37]:
t = target_standard_form(target_function, constraint_functions)
c = constraint_standard_form(constraint_functions)
c_matrix = c["matrix"]
constantes = c["constantes"]
v = variables(target_function)
nbr_split = len(v)

print("t = \n", t)
print("c = \n", c_matrix)
print("v = \n", v)
print("nbr split = ", nbr_split)
print("constantes = \n", constantes)
print("HELLO WORLD")

<=
i =  0
[-3.  2.  1.  0.  0.]
<=
i =  1
[-1.  2.  0.  1.  0.]
<=
i =  2
[1. 1. 0. 0. 1.]
t = 
 [-1, -2, 0, 0, 0]
c = 
 [[-3.  2.  1.  0.  0.]
 [-1.  2.  0.  1.  0.]
 [ 1.  1.  0.  0.  1.]]
v = 
 ['x1', 'x2']
nbr split =  2
constantes = 
 [[2], [4], [5]]
HELLO WORLD


In [38]:
def split_matrix(t,M):
    nbr_split = M.shape[0]
    
    base, hors_base = np.split(M, [nbr_split], axis=1)
    cb, cn = np.split(t, [nbr_split])  
    
    output = {
        "cb":cb,
        "cn":cn,
        "base":base,
        "hors_base":hors_base
    }
    
    return output
    
simplex_matrix = split_matrix(t,c_matrix)

print("CB = \n", simplex_matrix["cb"])
print("CN = \n", simplex_matrix["cn"])
print("Base = \n", simplex_matrix["base"])
print("Hors Base = \n", simplex_matrix["hors_base"])

CB = 
 [-1 -2  0]
CN = 
 [0 0]
Base = 
 [[-3.  2.  1.]
 [-1.  2.  0.]
 [ 1.  1.  0.]]
Hors Base = 
 [[0. 0.]
 [1. 0.]
 [0. 1.]]


In [63]:
from main import simplex

k=1
s = simplex(simplex_matrix["base"],simplex_matrix["hors_base"],
            np.array([simplex_matrix["cb"]]),np.array([simplex_matrix["cn"]]),
           constantes, k)

print("s = \n", s)

--------------ENTER-------NEW--------
k =  1
Base = 
 [[-3.  2.  1.]
 [-1.  2.  0.]
 [ 1.  1.  0.]]
N = 
 [[0. 0.]
 [1. 0.]
 [0. 1.]]
CN =  [[0 0]]
CB =  [[-1 -2  0]]
B_barre = 
 [[2.]
 [3.]
 [2.]]
PI =  [[-5.18104078e-17 -3.33333333e-01 -1.33333333e+00]]
CN barre =  [[0.33333333 1.33333333]]
Check CN barre =  True
B_barre solution = 
 [[2.]
 [3.]
 [2.]]
s = 
 [[2.]
 [3.]
 [2.]]


In [40]:
arr = [-1, -2, 0]
np.array([arr])

array([[-1, -2,  0]])

In [41]:
def final_response(v, s): 
    response_tab = []
    for i in range(len(v)):
        response_tab.append(s[i][0])
    return response_tab

f = final_response(v, s)

In [42]:
def solution_print(phr, v, response_tab):
    for variable, resp in zip(v, response_tab):
        phr = phr.replace(variable, str(resp))
    
    sol = "phrase = ",phr,"= ",eval(phr)
    return sol

s_p = solution_print(target_function, v, f)
print(s_p)

('phrase = ', '1*2.0 +2*3.0', '= ', 8.0)


In [62]:
const_s = constraint_simple.split('\n')

def check_constraint(c_s,v, resp):
    for vari, resp in zip(v, resp):
        c_s = c_s.replace(vari, str(resp))
    return c_s

c_s = check_constraint(constraint_simple, v, f)

print("c_s = ", c_s)

def check_c(tab):
    tab_split = tab.split("\n")
    check = True
    for c in tab_split:
        if not eval(c):
            check = False
            break
    
    return check

ch_s = check_c(c_s)
print("check = ", ch_s)

c_s =  2.0 >= 0
3.0 >= 0
check =  True
